### Team Members Name (eid):
1. Ko Choi (kec788)
2. Mandeep Burdak (msb4384)
3. Nishant Kushwaha (nsk779)
4. Pratyush Sharma (ps35484)
5. Vaibhav Nagar (vn5339)

### 1. Scraper code

In [0]:
import sys
import os
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.common.by import By
from tqdm import tqdm
import pandas as pd
from random import randint

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string
import statsmodels.api as sm
from nltk import FreqDist
from nltk.corpus import stopwords
from tqdm import tqdm
from sklearn.manifold import MDS
from matplotlib import pyplot as plt

In [0]:
path = '/mnt/car_brand_attribute_associations/'

In [0]:
wd = webdriver.Chrome(options=chrome_options)

n = 6000
data = {}
user_dict = {}
posts_dict = {}
time_dict = {}
i = 435
while len(data) < n:
    url = 'https://forums.edmunds.com/discussion/2864/general/x/entry-level-luxury-performance-sedans' + "/p" + str(i)
    wd.get(url)

    comments = wd.find_elements(By.CLASS_NAME, value="Comment")

    for comment in reversed(comments): 
        user = comment.find_element(By.TAG_NAME, "a").get_attribute('href')
        user = user.split('/')[-1]
        
        posts = comment.find_element(by=By.CLASS_NAME, value="AuthorInfo")
        posts = posts.find_element(by=By.TAG_NAME, value="b").text
        posts = posts.split(',')
        posts = "".join(posts)
        
        time = comment.find_element(by=By.TAG_NAME, value="time").get_attribute("datetime")

        try:
            backquote = comment.find_element(by=By.CLASS_NAME, value="Item-BodyWrap")
            backquote = backquote.find_element(by=By.TAG_NAME, value="blockquote")
            backquote = backquote.find_element(by=By.CLASS_NAME, value="QuoteText.blockquote-content").text
            backquote = backquote.split()
            backquote = " ".join(backquote)
            backquote = backquote.strip()
        except:
            backquote = ""
        
        comment_text = comment.find_element(by=By.CLASS_NAME, value="Item-BodyWrap")
        comment_text = comment.find_element(by=By.CLASS_NAME, value="Message.userContent").text
        comment_text = comment_text.split()
        comment_text = " ".join(comment_text)
        comment_text = comment_text.strip()

        try:
            sub_text = comment.find_element(by=By.CLASS_NAME, value="Item-BodyWrap")
            sub_text = sub_text.find_element(by=By.TAG_NAME, value="blockquote").text
            sub_text = sub_text.split()
            sub_text = " ".join(sub_text)
            sub_text = sub_text.strip()
        except:
            sub_text = ''

        comment_text = comment_text.replace(sub_text, "").strip()

        user_dict[comment_text] = user
        posts_dict[comment_text] = posts
        time_dict[comment_text] = time

        if backquote != "":
            if comment_text in data.keys():
                text = [comment_text]
                for j in data[comment_text]:
                    text.append(j)
                data[backquote] = text
                del data[comment_text]
            else:
                data[backquote] = [comment_text]
        else:
            if comment_text not in data.keys():
                data[comment_text] = []

    l = 30
    d = int((len(data)/n) * l)
    sys.stdout.write("\rCollecing samples from {1}: {0}>".format("="*d + "."*(l-d-1), url))
    sys.stdout.flush()
    i = int(i)
    i = i - 1
wd.quit()

In [0]:
edmunds = []
for key in data.keys():
    text = ' '
    text += key
    for j in data[key]:
        text += ' ' + j
    text.strip()

    try:
        user = user_dict[key]
    except KeyError:
        user = None
    
    try: 
        n_posts = posts_dict[key]
    except KeyError:
        n_posts = None

    try:
        time = time_dict[key]
    except KeyError:
        time = None
    
    edmunds.append([user, time, text, n_posts])

edmunds = pd.DataFrame(edmunds, columns=['User', 'Post Time', 'Comment', '#Posts by User'])
edmunds['Post Time'] = pd.to_datetime(edmunds['Post Time'])
edmunds = edmunds.loc[~edmunds['Comment'].isnull()]
edmunds = edmunds.loc[~edmunds['Post Time'].isnull()]
edmunds = edmunds[:5000]
df_spark = spark.createDataFrame(edmunds)
df_spark.write.option("header", "true").mode("overwrite").csv(path+'Data/Edmunds_Data_New')
edmunds.info()

### OLS- Zipf's Law verification

In [0]:
edmunds_data = edmunds
edmunds_data.info()

In [0]:
def generate_corpus(column):
    corpus = " "
    for text in column:
        text = str(text)
        for i in list(string.punctuation.replace('+','')):
            text = text.replace(i, ' ')
        text = text.lower()

        corpus += text
    return corpus

In [0]:
corpus = generate_corpus(edmunds_data['Comment'])
corpus = corpus.split()
print(len(corpus))

word_freq = FreqDist()
for word in tqdm(corpus):
    word_freq[word] += 1

freq_dist = []
for rank, word in enumerate(word_freq):
    freq_dist.append([rank+1, word, word_freq[word]])

freq_dist = pd.DataFrame(freq_dist, columns=['Rank', 'Word', 'Count'])
df = spark.createDataFrame(freq_dist)
df.write.option("header", "true").mode("overwrite").csv(path+ 'Support Files/Freq_Dist_new_comments')
freq_dist

In [0]:
max_rank = max(freq_dist['Rank'])
c = max_rank * int(freq_dist.loc[freq_dist['Rank'] == max_rank]['Count'].values)

l_rank = np.log(freq_dist['Rank'] / c)
l_freq = np.log(freq_dist['Count'])

# X = sm.add_constant(l_rank)
model = sm.OLS(l_freq, l_rank)
result = model.fit()
print(result.summary())

In [0]:
plt.figure(figsize=(20,7))
plt.plot(l_freq, l_rank, label="Raw Zipf's Curve")
plt.plot(result.predict(l_rank), l_rank, label='OLS Fit')
plt.xlabel('Rank')
plt.ylabel('Frequency')
plt.title("Zipf's Distribution")
plt.legend()
plt.show()

### 2. Which 10 brands you chose – provide the frequency table

In [0]:
def process_column(column):
    data = list(column)
    for i in range(len(data)):
        text = str(data[i])
        for punct in list(string.punctuation.replace('+','')):
             text = text.replace(punct, ' ')
        text = text.lower()
        data[i] = text
    return data

def replace_car_brands(data, map):
    for i in range(len(data)):
        text = data[i]
        l_text = text.split()
        for word in l_text:
            if word in map.keys():
                text = text.replace(word, map[word])
        data[i] = text
    return data

In [0]:
comments = process_column(edmunds_data['Comment'])

df = spark.read.format('csv').options(header=True,inferSchema=True).load(path + 'Support Files/Substitutions/')
brand_model_map = df.toPandas()

model__brand_dict = {}
for brand, model in zip(brand_model_map['Brand'], brand_model_map['Model']):
    model__brand_dict[model] = brand

# print(model__brand_dict['century'])
comments = replace_car_brands(comments, model__brand_dict)

brand_count = {}
for brand in list(brand_model_map['Brand'].unique()):
    brand_count[brand] = 0

for i in range(len(comments)):
    text = comments[i].split()
    for brand in list(brand_count.keys()):
        if brand in text:
            brand_count[brand] += 1


In [0]:
brand_freq = []
for brand, count in zip(brand_count.keys(), brand_count.values()):
    brand_freq.append([brand, count])

brand_freq = pd.DataFrame(brand_freq, columns=['Brand', 'Count'])
brand_freq = brand_freq.sort_values('Count', ascending=False).reset_index(drop=False)
brand_freq

### 3. Show all lift values in a table 

In [0]:
def lift(texts, x, y, words=float('inf')):
    n = len(texts)
    count_x = 0
    count_y = 0
    count_x_y = 0

    for i in range(n):
        x_indices = []
        y_indices = []
        text = texts[i].split()
        
        for j in range(len(text)):
            if x == text[j]:
                x_indices.append(j)
            elif y == text[j]:
                y_indices.append(j)

        n_words = []    
        for x_index in x_indices:
            for y_index in y_indices:
                n_words.append(abs(x_index - y_index) - 1)
        
        # print(len(n_words) > 0, len(x_indices) > 0, len(y_indices) > 0)
        if len(n_words) > 0:
            if float(min(n_words)) <= float(words):
                count_x_y += 1
        if len(x_indices) > 0:
            count_x += 1
        if len(y_indices) > 0:
            count_y += 1
        
        # print(n, count_x_y, count_x, count_y)
    
    lift = (n * count_x_y) / (count_x * count_y)
    return lift

In [0]:
top_10_brands = list(brand_freq[:10]['Brand'])

lift_scores = []
dissimilarity_matrix = []
for i in range(len(top_10_brands)):
    row = []
    row_diss = []
    for j in range(len(top_10_brands)):
        if j!=i:
            lift_score = lift(comments, top_10_brands[i], top_10_brands[j])
            row.append(lift_score)
            try:
                row_diss.append(1/lift_score)
            except ZeroDivisionError:
                row_diss.append(np.inf)
        elif i==j:
            row.append(1)
            row_diss.append(0)
        # else:
        #     row.append(np.nan)
        #     row_diss.append(np.nan)
    
    row.append(top_10_brands[i])
    row_diss.append(top_10_brands[i])
    lift_scores.append(row)
    dissimilarity_matrix.append(row_diss)

cols = top_10_brands.append('Brand')

lift_scores = pd.DataFrame(lift_scores, columns=top_10_brands).set_index('Brand')
display(lift_scores)

dissimilarity_matrix = pd.DataFrame(dissimilarity_matrix, columns=top_10_brands).set_index('Brand')
display(dissimilarity_matrix)

### 4. MDS Map

In [0]:
def MDS_map(dissimilarity_matrix, metric=True, title='MDS Plot'):
    mds = MDS(n_components=2, metric=metric, dissimilarity='precomputed', random_state=0)
    pts = mds.fit_transform(dissimilarity_matrix)

    plt.scatter(pts[:,0], pts[:,1], color='silver', s=150)
    for i in range(dissimilarity_matrix.shape[0]):
        plt.annotate(dissimilarity_matrix.index[i], (pts[i,0], pts[i,1]), color='blue')
    plt.title(title)
    plt.axis('off')
    plt.show()

MDS_map(dissimilarity_matrix=dissimilarity_matrix, metric=True, title='MDS Plot - Car Brands')

### 5. State the 5 attributes you chose


![Screenshot 2023-09-12 at 6.09.07 PM.png](<attachment:Screenshot 2023-09-12 at 6.09.07 PM.png>)

The image above illustrates the top words we were able to identify for attributes. The remaining words consisted mainly of punctuation marks or were otherwise not relevant.

In [0]:
# features = input("Enter the features as comma seperated values without spaces")
# features = features.split(',')
brand_feature = list(brand_freq[:10]['Brand'])
features = ['performance', 'luxury', 'driving', 'engine', 'handling', 'interior']
for feature in features:
    brand_feature.append(feature)


lift_scores = []
dissimilarity_matrix = []
for i in range(len(brand_feature)):
    row = []
    row_diss = []
    for j in range(len(brand_feature)):
        if j!=i:
            if (brand_feature[i] in top_10_brands) and (brand_feature[j] in top_10_brands):
                lift_score = lift(comments, brand_feature[i], brand_feature[j])
            else:
                lift_score = lift(comments, brand_feature[i], brand_feature[j], 40)
            row.append(lift_score)
            try:
                row_diss.append(1/lift_score)
            except ZeroDivisionError:
                row_diss.append(np.inf)
        elif i==j:
            row.append(1)
            row_diss.append(0)
        # else:
        #     row.append(np.nan)
        #     row_diss.append(np.nan)
    
    row.append(brand_feature[i])
    row_diss.append(brand_feature[i])
    lift_scores.append(row)
    dissimilarity_matrix.append(row_diss)

cols = brand_feature.append('Brand')

lift_scores = pd.DataFrame(lift_scores, columns=brand_feature).set_index('Brand')
display(lift_scores)

dissimilarity_matrix = pd.DataFrame(dissimilarity_matrix, columns=brand_feature).set_index('Brand')
display(dissimilarity_matrix)

#### 6. For task F, provide all details of your analysis – e.g., how you measured “aspirational” and how you found the most aspirational brand

![Screenshot 2023-09-12 at 5.27.41 PM.png](<attachment:Screenshot 2023-09-12 at 5.27.41 PM.png>)

As you can see in the image above, we are evaluating the Lyft value for each brand against their respective features. Whenever Lyft is greater than 1 for a particular feature, we assign it a score of 1; otherwise, we assign a score of 0. 
Based on this analysis, Audi seems to be the most aspirational brand, while for luxury, Lexus and Cadillac stand out as excellent choices!

### 7. Advice/insights based on your analysis for your client

Advice/insights based on your analysis for your client
Based on the analysis of brand preferences within luxury segments and the relationships between different automotive brands, here are some advice and insights for your client:

*1. Brand Synergy and Cross-Promotion:*
   - Clients who prefer luxury brands like Lexus, Mercedes, and Cadillac tend to have similar preferences. Encourage cross-promotion or partnerships between these brands to capture a broader customer base.
   - Consider joint marketing efforts or collaborations to leverage the shared appeal among these luxury brands.

*2. Leveraging the German Luxury Cluster:*
   - Given the high similarity values between BMW, Audi, and Mercedes, target marketing strategies towards consumers who appreciate German luxury engineering and design.
   - Highlight the strengths and unique features of each brand within this cluster to differentiate them while appealing to the shared preferences.

*3. Exploring Japanese Brand Synergy:*
   - Recognize that Toyota and Honda share a moderate level of similarity, suggesting some customer overlap. Develop marketing initiatives or offers that cater to consumers who prefer both Japanese brands.
   - For Acura, focus on its unique value proposition and characteristics to target a slightly different customer base within the Japanese automaker market.

*4. Understanding Infiniti's Niche:*
   - Infiniti's relative isolation indicates a unique market position or a different target audience. Conduct market research to identify the specific consumer segment that Infiniti appeals to and tailor marketing efforts accordingly.
   - Highlight the distinctive qualities of Infiniti vehicles to attract and retain this niche customer base.

*5. Volkswagen's Versatility:*
   - Volkswagen's moderate similarity with various brands suggests that it occupies a middle ground between different car segments. Leverage this versatility to appeal to a broader range of consumers.
   - Emphasize Volkswagen's adaptability, offering vehicles that can meet a variety of customer needs and preferences.

*6. Cadillac's Competitive Landscape:*
   - Cadillac's lower similarity with other American brands and higher correlations with European luxury brands suggest that it competes more directly with European luxury brands.
   - Position Cadillac as a strong contender in the European luxury segment and emphasize the qualities that make it competitive within this category.

*7. Capitalize on the Lexus-Toyota Connection:*
   - The strong similarity between Lexus and Toyota indicates a strong connection between the two brands. Leverage Toyota's reputation for reliability to attract consumers to the Lexus luxury offerings.
   - Highlight the shared commitment to quality and reliability in marketing campaigns for both brands.

In conclusion, understanding the relationships and preferences among luxury automotive brands can guide your client's marketing and strategic decisions. Tailoring marketing efforts to capitalize on brand synergy, targeting specific customer segments, and emphasizing unique brand qualities are key strategies to maximize success in the competitive luxury automobile market.